In [ ]:
Periods: 2015,2019
Source: http://imd-by-postcode.opendatacommunities.org/imd/2015 or 2019

In [ ]:
import pandas as pd
import re
file_names=['2015-deprivation-by-postcode.xlsx','2019-deprivation-by-postcode.xlsx']
years_dfs={}
for file_name in file_names:
    year=re.findall('\d{4}',file_name)[0]
    print (year)
    data_path='/Users/rony/Projects/Health_Geo_app/data/raw_data/deprivation/'
    deprivation=pd.read_excel('{dp}{fn}'.format(dp=data_path,fn=file_name))
    locations=pd.read_csv('../../data/location_data.csv')
    deprivation=deprivation.drop(columns=['Postcode Status','LSOA Name','LSOA code'])
    practices_locations=locations[['Practice_code','Postcode']]
    deprivation_data=pd.merge(practices_locations,deprivation)
    deprivation_data=deprivation_data.drop_duplicates()
    del deprivation_data ['Postcode']
    print (deprivation_data.info())
    years_dfs[year]=deprivation_data

In [31]:
years_dfs['2015'].iloc[:,:5].head()

,Practice_code,Index of Multiple Deprivation Rank,Index of Multiple Deprivation Decile,Income Rank,Income Decile
0,A81002,551,1,433,1
1,A81004,22673,7,21470,7
2,A81005,7082,3,5343,2
4,A81032,7082,3,5343,2
6,A81006,551,1,433,1


In [32]:
years_dfs['2019'].iloc[:,:5].head()

,Practice_code,Index of Multiple Deprivation Rank,Index of Multiple Deprivation Decile,Income Rank,Income Decile
0,A81002,229,1,145,1
1,A81004,22602,7,20854,7
2,A81005,7460,3,4654,2
4,A81032,7460,3,4654,2
6,A81006,229,1,145,1


In [33]:
# Melt dataframes
years_melted_dfs={}
for year,df in years_dfs.items():    
    # Melt the annual data files to the new format which reflect the query UI in terms of separte gender and age
    dep_melt=pd.melt(frame=df, id_vars='Practice_code', value_name='Value')
    variables=list(dep_melt['variable'])
    dep_melt['Measure'] = dep_melt['variable'].map(lambda v:re.findall('Rank|Decile|Score',v)[0].rstrip().lstrip())
    dep_melt['Deprivation_Index']= [re.sub(dep_melt['Measure'][ind],'',v).rstrip().lstrip().replace(' ','_')\
                       for ind,v in enumerate (variables)]
    dep_melt['Year']=year
    dep_melt=dep_melt[['Practice_code', 'Deprivation_Index','Year','Measure', 'Value']]
    years_melted_dfs[year]=dep_melt

In [35]:
years_melted_dfs['2015'].head()

,Practice_code,Deprivation_Index,Year,Measure,Value
0,A81002,Index_of_Multiple_Deprivation,2015,Rank,551.0
1,A81004,Index_of_Multiple_Deprivation,2015,Rank,22673.0
2,A81005,Index_of_Multiple_Deprivation,2015,Rank,7082.0
3,A81032,Index_of_Multiple_Deprivation,2015,Rank,7082.0
4,A81006,Index_of_Multiple_Deprivation,2015,Rank,551.0


In [36]:
years_melted_dfs['2019'].head()

,Practice_code,Deprivation_Index,Year,Measure,Value
0,A81002,Index_of_Multiple_Deprivation,2019,Rank,229.0
1,A81004,Index_of_Multiple_Deprivation,2019,Rank,22602.0
2,A81005,Index_of_Multiple_Deprivation,2019,Rank,7460.0
3,A81032,Index_of_Multiple_Deprivation,2019,Rank,7460.0
4,A81006,Index_of_Multiple_Deprivation,2019,Rank,229.0


In [54]:
measures_split={}
results_path='/Users/rony/Projects/Health_Geo_app/data/deprivation/'
for year,dep_melt in years_melted_dfs.items():    
    deciles_df=dep_melt[dep_melt['Measure']=='Decile']
    deciles_df=deciles_df.rename(columns={'Measure':'Decile'})
    deciles_df['Decile']=deciles_df['Value']
    del deciles_df['Value']
    ranks_df=dep_melt[dep_melt['Measure']=='Rank']
    deciles_df['Rank']=list(ranks_df['Value'])
    deciles_df[['Decile','Rank']]=deciles_df[['Decile','Rank']].astype(int)
    file_name='deprivation_{y}'.format (y=year)
    deciles_df.to_csv('{rp}{fn}.csv'.format(rp=results_path,fn=file_name), index=False)
    measures_split[year]=deciles_df

In [55]:
measures_split['2015'].head(20)

,Practice_code,Deprivation_Index,Year,Decile,Rank
6970,A81002,Index_of_Multiple_Deprivation,2015,1,551
6971,A81004,Index_of_Multiple_Deprivation,2015,7,22673
6972,A81005,Index_of_Multiple_Deprivation,2015,3,7082
6973,A81032,Index_of_Multiple_Deprivation,2015,3,7082
6974,A81006,Index_of_Multiple_Deprivation,2015,1,551
6975,A81007,Index_of_Multiple_Deprivation,2015,1,967
6976,A81011,Index_of_Multiple_Deprivation,2015,1,967
6977,A81031,Index_of_Multiple_Deprivation,2015,1,967
6978,A81009,Index_of_Multiple_Deprivation,2015,1,2155
6979,A81012,Index_of_Multiple_Deprivation,2015,1,59


In [57]:
measures_split['2019'].head(20)

,Practice_code,Deprivation_Index,Year,Decile,Rank
6970,A81002,Index_of_Multiple_Deprivation,2019,1,229
6971,A81004,Index_of_Multiple_Deprivation,2019,7,22602
6972,A81005,Index_of_Multiple_Deprivation,2019,3,7460
6973,A81032,Index_of_Multiple_Deprivation,2019,3,7460
6974,A81006,Index_of_Multiple_Deprivation,2019,1,229
6975,A81007,Index_of_Multiple_Deprivation,2019,1,430
6976,A81011,Index_of_Multiple_Deprivation,2019,1,430
6977,A81031,Index_of_Multiple_Deprivation,2019,1,430
6978,A81009,Index_of_Multiple_Deprivation,2019,1,1292
6979,A81012,Index_of_Multiple_Deprivation,2019,1,138


In [56]:
#ranks only
ranks={'2015':list(measures_split['2015']['Rank']),'2019':list(measures_split['2019']['Rank'])}
ranks_df=pd.DataFrame.from_dict(ranks)
ranks_df.head(20)

,2015,2019
0,551,229
1,22673,22602
2,7082,7460
3,7082,7460
4,551,229
5,967,430
6,967,430
7,967,430
8,2155,1292
9,59,138
